# Preliminary Investigation of Avalanche Features
The Utah Avalanche Center has a [dataset](https://utahavalanchecenter.org/avalanches) containing reports of avalanches dating back to 1914. While it isn't comprehensive, I believe it can be used as part of a collection of datasets to predict avalanches. Just so I can get more intimate with this data set, in this notebook I'll hone in on five attributes that were recorded:

1) [Region of avalanche](#region)  
2) [Day of week of avalanche](#day)  
3) [Elevation of avalanche](#elevation)  
4) [Aspect of avalanche](#aspect)  
5) [Month of avalanche](#month)  

For each of these, I'll look into both the number of deaths, the number of avalanches, and the number of deaths per the number of avalanches to see which levels of these attributes are most dangerous.

In [114]:
import sqlalchemy
sqlalchemy.create_engine('postgresql://john@localhost/avy')
%load_ext sql
%sql postgresql:///avy

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @avy'

<a name="region"></a>
## 1) Region of avalanche
There are ten regions included in the dataset: Salt Lake, Provo, Skyline, Uintas, Ogden, Logan, Moab, Abajos, SE Idaho, and Southwest. A fatality has never been recorded in the Abajos region and only one in the SE Idaho region but no coordinates were provided. That's why the Abajos and SE Idaho regions aren't in the legend for the map.

In the following graphic, we can see that most observed avalanches occur in Salt Lake. However, the Salt Lake region has the second lowest number of deaths per avalanche reports: 2.09 per 100 avalanche reports. 

At first glance, perhaps the Southwest region looks the most dangerous. But there's only 17 recorded avalanches for the Southwest region, so the sample size is to small to draw firm conclusions in this regard. The Ogden region seems a more likely candidate for the most dangerous region with 5.47 deaths per 100 reported avalanches. That's over twice as many deaths per avalanche reportings as Salt Lake.

In [35]:
%%html
<div class='tableauPlaceholder' id='viz1588781913759' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Av&#47;AvalancheDangerbyRegion&#47;deathRatio&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='AvalancheDangerbyRegion&#47;deathRatio' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Av&#47;AvalancheDangerbyRegion&#47;deathRatio&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1588781913759');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.minWidth='420px';vizElement.style.maxWidth='850px';vizElement.style.width='100%';vizElement.style.minHeight='587px';vizElement.style.maxHeight='887px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.minWidth='420px';vizElement.style.maxWidth='850px';vizElement.style.width='100%';vizElement.style.minHeight='587px';vizElement.style.maxHeight='887px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else { vizElement.style.width='100%';vizElement.style.height='777px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

<a name="day"></a>
## 2) Day of Week of Avalanche
Without human activity, avalanches occur at the same rate regardless of the day of the week. But there's increased traffic on the weekends and maybe that causes avalanches to occur more. In the following SQL printouts we can see the breakdown of avalanches by day of the week. Both avalanche observations and fatalities occur more frequently on the weekends. Of note: Tuesday sees the least amount of fatalities by quite a large margin: Only 5.7% of all recorded avalanche fatalities in Utah have happened on Tuesdays.

I didn't include the printout for brevity, but of the 26 avalanche fatalities that have occured since the 2009/2010 season, none have occurred on a Monday or Tuesday.

In [126]:
%%sql

SELECT x.day_alpha AS day_of_week, x.avy_count AS avy_county, x.per AS percentage FROM (
SELECT to_char(date, 'Day') AS day_alpha, date_part('isodow', date) AS day_numeric, 
    COUNT(*) AS avy_count, ROUND(100 * COUNT(*) / SUM(COUNT(*)) OVER(), 1) AS per
FROM avy
GROUP BY date_part('isodow', date), to_char(date, 'Day')
ORDER BY date_part('isodow', date)) AS x;

 * postgresql:///avy
7 rows affected.


day_of_week,avy_county,percentage
Monday,612,12.4
Tuesday,549,11.1
Wednesday,616,12.5
Thursday,620,12.6
Friday,756,15.3
Saturday,939,19.0
Sunday,844,17.1


And also, the breakdown of fatalities over the days of the week:

In [110]:
%%sql

SELECT x.day_alpha AS day_of_week, x.fat_count AS fatality_county, x.per AS percentage FROM (
SELECT to_char(date, 'Day') AS day_alpha, date_part('isodow', date) AS day_numeric, 
    COUNT(*) AS fat_count, ROUND(100 * COUNT(*) / SUM(COUNT(*)) OVER(), 1) AS per
FROM avy
WHERE death > 0
GROUP BY date_part('isodow', date), to_char(date, 'Day')
ORDER BY date_part('isodow', date)) AS x;

 * postgresql:///avy
7 rows affected.


day_of_week,fatality_county,percentage
Monday,10,9.4
Tuesday,6,5.7
Wednesday,10,9.4
Thursday,13,12.3
Friday,18,17.0
Saturday,32,30.2
Sunday,17,16.0


<a name='elevation'></a>
## 3) Elevation of avalanche
No big surprises here. The first two bar graphs show fatalities across different elevations. I was also curious to see what portion of avalanches at a given elevation occur over the course of the winter. For example, I wondered if avalanches below 7,000 feet only occur in the middle of winter. This seems to be true: if the months April, May, June, October, and November are selected from the legend (`ctrl`-click to multi-select) then we can see that, proportionally, very few avalanches occur in general but hardly any occur below 7,000 feet.

Also, it's just kinda interesting to mountain nerds that there've only been 8 avalanche reportings above 12,000 ft in Utah. If we look, we see that they're all in the Moab region which means they were in the La Sal mountain range.

In [46]:
%%html
<div class='tableauPlaceholder' id='viz1588784111079' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Av&#47;AvalancheDangerbyElevation&#47;avy_deathElev&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='AvalancheDangerbyElevation&#47;avy_deathElev' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Av&#47;AvalancheDangerbyElevation&#47;avy_deathElev&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1588784111079');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.minWidth='420px';vizElement.style.maxWidth='850px';vizElement.style.width='100%';vizElement.style.minHeight='587px';vizElement.style.maxHeight='887px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.minWidth='420px';vizElement.style.maxWidth='850px';vizElement.style.width='100%';vizElement.style.minHeight='587px';vizElement.style.maxHeight='887px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else { vizElement.style.width='100%';vizElement.style.height='977px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

In [112]:
%%sql

SELECT *
FROM avy
WHERE elev_ft >= 12000;

 * postgresql:///avy
8 rows affected.


avy_id,date,region,depth_in,width_ft,vert_ft,aspect,elev_ft,caught,carried,buried_partial,buried_full,injured,death,lat,lon
840,2019-01-25,Moab,48,900,1500,East,12200,1,1,0,1,0,1,38.456579716615,-109.23708199209
2412,2015-01-17,Moab,48,70,1750,Northeast,12100,0,0,0,0,0,0,38.447261052346,-109.26396715836
3008,2013-12-07,Moab,36,350,600,Northeast,12000,0,0,0,0,0,0,38.441432,-109.261385
4019,2011-02-08,Moab,None,None,None,North,12200,0,0,0,0,0,0,None,None
4098,2011-01-11,Moab,24,50,100,North,12200,0,0,0,0,0,0,38.464813,-109.234486
4163,2010-12-23,Moab,48,5000,None,North,12400,0,0,0,0,0,0,38.44001,-109.229422
4322,2010-03-14,Moab,24,None,1000,North,12200,0,0,0,0,0,0,38.440951,-109.259248
4485,2010-02-20,Moab,54,3500,None,Northeast,12000,0,0,0,0,0,0,38.455807,-109.239292


<a name='aspect'></a>
## 4) Aspect
It's generally know that north and eastern facing slopes see less sun, hold more snow, and are a greater avalanche hazard. This can be seen in the following graphic. I was also curious about the distribution of the depth of avalanches across aspects. It does appear that northwest aspects are more dangerous, per avalanche (look at the bottom bar graph) but I don't have an explanation for this.

In [115]:
%%html
<div class='tableauPlaceholder' id='viz1588791849610' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Av&#47;AvalancheDangerbyAspect&#47;byasp&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='AvalancheDangerbyAspect&#47;byasp' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Av&#47;AvalancheDangerbyAspect&#47;byasp&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1588791849610');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.minWidth='420px';vizElement.style.maxWidth='850px';vizElement.style.width='100%';vizElement.style.minHeight='587px';vizElement.style.maxHeight='887px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.minWidth='420px';vizElement.style.maxWidth='850px';vizElement.style.width='100%';vizElement.style.minHeight='587px';vizElement.style.maxHeight='887px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else { vizElement.style.width='100%';vizElement.style.height='977px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

<a name='month'></a>
## 5) Month
This attribute also holds no surprises. Everyone knows that avalanches happen between the start of the season and begin to chill out in April. That's what we see here. 

In [48]:
%%html
<div class='tableauPlaceholder' id='viz1588787269632' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Av&#47;AvalancheDangerbyMonth&#47;avy_dangMonth&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='AvalancheDangerbyMonth&#47;avy_dangMonth' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Av&#47;AvalancheDangerbyMonth&#47;avy_dangMonth&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1588787269632');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.minWidth='420px';vizElement.style.maxWidth='850px';vizElement.style.width='100%';vizElement.style.minHeight='587px';vizElement.style.maxHeight='887px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.minWidth='420px';vizElement.style.maxWidth='850px';vizElement.style.width='100%';vizElement.style.minHeight='587px';vizElement.style.maxHeight='887px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else { vizElement.style.width='100%';vizElement.style.height='877px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>